In [1]:
from fuzzingbook.GrammarFuzzer import GrammarFuzzer
from fuzzingbook.Grammars import Grammar
from debuggingbook.DeltaDebugger import DeltaDebugger
from debuggingbook.DDSetDebugger import DDSetDebugger
from exercise_1 import heartbeat

#### 1.

In [2]:
hearbeat_grammar: Grammar={"<start>":["<n>"], "<n>": ["<n><symbol>","<symbol>"], "<symbol>": ['a','b','c','d','e','f']}

#### 2. & 3.

In [3]:
G=GrammarFuzzer(hearbeat_grammar)
#for i in range(10):
fuzzed_input: str = G.fuzz()
print(fuzzed_input)
with DeltaDebugger() as dd:
    heartbeat(3,fuzzed_input)
print(dd.min_args())


a
{'length': 3, 'payload': ''}


The minimzed input is just an empty string, which is correct, but does not explain the failure reason properly.

In [7]:
with DDSetDebugger(hearbeat_grammar) as dds:
    heartbeat(3,fuzzed_input)
dds

heartbeat(length=3, payload='<n>')

#### 4.1

The pattern presented by DDSetDebugger is incorrect, because not all instantiations of this pattern will lead to failure of the heartbeat function.


In [9]:
fail:int =0
n: int =10000
for i in range(n):
    gen=dds.fuzz_args()
    a=gen['length']
    b=gen['payload']
    try:
        heartbeat(a,b)
    except AssertionError:
        fail=fail+1

print(f"Succes rate:{float(fail/n)}")

Succes rate:0.8803


#### 4.2
As succes rate is equal to around 0.86, it is not that high, we would expect it to be 100%, as the DDSetDebugger should produce only failing inputs

#### 4.3
To make DDSetDebugger more infomative, it should take into account how many terminals symbols can be produced from the given terminal. By testing the obtained pattern it could learn that the problem lies in the length of the payload